In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
import tensorflow as tf
import tensorflow.keras as keras
import os
import numpy as np
import matplotlib.pyplot as plt
import transformers 
from transformers import TFAutoModel, AutoTokenizer
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import pandas as pd 
import numpy as np 
from sklearn.model_selection import train_test_split

In [ ]:
df = pd.read_csv("../input/kwproject/kwprojb_data.csv")

In [ ]:
def Utilize_TPUs():  
    """
    Initialize training strategy using TPU if available else using default strategy for CPU and  single GPU
    
    After the TPU is initialized, you can also use manual device placement to place the computation on a single TPU device.

    """
    try:
        
        resolver = tf.distribute.cluster_resolver.TPUClusterResolver()
        tf.config.experimental_connect_to_cluster(resolver)
        tf.tpu.experimental.initialize_tpu_system(resolver)
        strategy = tf.distribute.experimental.TPUStrategy(resolver)
        REPLICAS = strategy.num_replicas_in_sync
        print("Connected to TPU Successfully:\n TPUs Initialised with Replicas:",REPLICAS)
        
        return strategy
    
    except ValueError:
        
        print("Connection to TPU Falied")
        print("Using default strategy for CPU and single GPU")
        strategy = tf.distribute.get_strategy()
        
        return strategy
    
strategy=Utilize_TPUs()

In [ ]:
#df = train_data_frame

In [ ]:
A = df[df['shop_type'] == '세탁소'].index

In [ ]:
df.drop(A,axis='index',inplace=True)
df

In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
df['shop_type'] = le.fit_transform(list(df['shop_type']))

In [ ]:
X_column = ["shop_name"]
y_column = 'shop_type'

In [ ]:
input_data = df[X_column + [y_column]].copy()
input_data

In [ ]:
#tk.texts_to_sequences(input_data['shop_name'])

In [ ]:
#import tensorflow as tf
#from tensorflow.keras.preprocessing.text import Tokenizer

#tk = Tokenizer() 
#tk.fit_on_texts(input_data['shop_name'])

In [ ]:
the_chosen_one="bert-base-cased"

#the_chosen_one="jplu/tf-xlm-roberta-base"

max_len = 8
batch_size = 16 * strategy.num_replicas_in_sync
AUTO = tf.data.experimental.AUTOTUNE
epochs = 20
n_steps = len(df) // batch_size

In [ ]:
def model_baseline(strategy,transformer):
    with strategy.scope():
        transformer_encoder = TFAutoModel.from_pretrained(transformer)
        input_layer = Input(shape=(max_len,), dtype=tf.int32, name="input_layer")
        sequence_output = transformer_encoder(input_layer)[0]
        cls_token = sequence_output[:, 0, :]
        output_layer = Dense(61, activation='softmax')(cls_token)
        model = Model(inputs=input_layer, outputs=output_layer)
        model.compile(
            Adam(lr=1e-5), 
            loss='sparse_categorical_crossentropy', 
            metrics=['accuracy']
        )
        return model
model=model_baseline(strategy,the_chosen_one)

In [ ]:
model.summary()

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(the_chosen_one)

In [ ]:
all_data = input_data['shop_name'].values.tolist()

In [ ]:
test_data = input_data['shop_name'].values.tolist()

In [ ]:
train_encoded=tokenizer.batch_encode_plus(train_data,pad_to_max_length=True,max_length=max_len)

In [ ]:
# 데이터 -> train/valid, test
from sklearn.model_selection import train_test_split

raw_tr_val_X, raw_test_X, tr_val_y, test_y = train_test_split(
    input_data[X_column], input_data[y_column],
    test_size=0.2, shuffle=True, random_state=42, stratify=input_data[y_column])


In [ ]:
# train/valid -> train, valid split
raw_train_X, raw_valid_X, train_y, valid_y = train_test_split(
    raw_tr_val_X, tr_val_y,
    test_size=0.2, shuffle=True, random_state=42, stratify=tr_val_y)

In [ ]:
train_data = raw_train_X['shop_name'].values.tolist()
valid_data = raw_valid_X['shop_name'].values.tolist()
test_data = raw_test_X['shop_name'].values.tolist()

In [ ]:
train_encoded = tokenizer.batch_encode_plus(train_data, pad_to_max_length=True, max_length=max_len)
valid_encoded = tokenizer.batch_encode_plus(valid_data, pad_to_max_length=True, max_length=max_len)
test_encoded = tokenizer.batch_encode_plus(test_data, pad_to_max_length=True, max_length=max_len)

In [ ]:
test_encoded=tokenizer.batch_encode_plus(test_data,pad_to_max_length=True,max_length=max_len)

In [ ]:
tr_val_X, test_X, tr_val_y, test_y = train_test_split(input_data[X_column], input_data[y_column], test_size=0.2, shuffle=True, random_state=42, stratify=input_data[y_column])

In [ ]:
train_X, valid_X, train_y, valid_y = train_test_split(tr_val_X, tr_val_y, test_size=0.2, shuffle=True, random_state=42, stratify=tr_val_y)

In [ ]:
x_train = train_encoded['input_ids']
x_valid = valid_encoded['input_ids']
x_test = test_encoded['input_ids']


In [ ]:
train_dataset = (tf.data.Dataset.from_tensor_slices((train_X, train_y)).repeat().shuffle(2048).batch(batch_size).prefetch(AUTO))

valid_dataset = (tf.data.Dataset.from_tensor_slices((valid_X, valid_y)).batch(batch_size).cache().prefetch(AUTO))

test_dataset = (tf.data.Dataset.from_tensor_slices(test_X).batch(batch_size))

In [ ]:
train_dataset = (tf.data.Dataset.from_tensor_slices((x_train, train_y)).repeat().shuffle(2048).batch(batch_size).prefetch(AUTO))

valid_dataset = (tf.data.Dataset.from_tensor_slices((x_valid, valid_y)).batch(batch_size).cache().prefetch(AUTO))

test_dataset = (tf.data.Dataset.from_tensor_slices((x_test, test_y)).batch(batch_size).cache().prefetch(AUTO))


In [ ]:
#model.fit(train_X, train_y, validation_data=(valid_X, valid_y),epochs = epochs)

In [ ]:
hist = model.fit(train_dataset,steps_per_epoch=n_steps,validation_data=valid_dataset,epochs=epochs)

In [ ]:
pred_y_prob = model.predict(test_dataset, verbose=1)

In [ ]:
pred_y = np.argmax(pred_y_prob, axis=1)
pred_y

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

print(classification_report(test_y, pred_y))